In [147]:


import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

columns = ['id','id2','task','TaskId','rating']
frame = pd.read_csv('DataRecommenderSystem/DataDeliveryVisitor.csv',sep=',', names=columns)
frame.head()

,id,id2,task,TaskId,rating
0,1,2,1000,1,7
1,2,2,1010,2,8
2,3,2,101010,3,7
3,4,2,100010,4,5
4,5,2,1000,5,6


In [148]:
columns2 =['TaskId','TaskName','Task']
tasks = pd.read_csv('DataRecommenderSystem/TaskDeliveryVisitor.csv', sep=',', names = columns2, encoding='latin-1')
task_names = tasks[['TaskId','TaskName','Task']]
task_names.head()


,TaskId,TaskName,Task
0,1,Task1,1000
1,2,Task2,1010
2,3,Task3,101010
3,4,Task4,100010
4,5,Task5,1000


In [149]:
combined_task_data = pd.merge(frame, task_names, on='TaskId')
combined_task_data.head()

,id,id2,task,TaskId,rating,TaskName,Task
0,1,2,1000,1,7,Task1,1000
1,11,2,1000,1,9,Task1,1000
2,12,2,1000,1,5,Task1,1000
3,14,2,1000,1,4,Task1,1000
4,15,2,1000,1,9,Task1,1000


In [150]:
combined_task_data.groupby('TaskId')['rating'].count().sort_values(ascending=True).head()

filter_interview = combined_task_data['TaskId']==0
combined_task_data[filter_interview]['TaskName'].unique()

array([], dtype=object)

In [151]:
rating_crosstab = combined_task_data.pivot_table(values='rating', index='id', columns=['TaskName','Task'], fill_value=0.01)
rating_crosstab

TaskName,Task1,Task2,Task3,Task4,Task5,Task6,Task7,Task8
Task,1000,1010,101010,100010,1000,100000,1100,1111110
id,,,,,,,,
1,7.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01
2,0.01,8.00,0.01,0.01,0.01,0.01,0.01,0.01
3,0.01,0.01,7.00,0.01,0.01,0.01,0.01,0.01
4,0.01,0.01,0.01,5.00,0.01,0.01,0.01,0.01
5,0.01,0.01,0.01,0.01,6.00,0.01,0.01,0.01
...,...,...,...,...,...,...,...,...
97,0.01,5.00,0.01,0.01,0.01,0.01,0.01,0.01
98,0.01,0.01,0.01,0.01,9.00,0.01,0.01,0.01


In [152]:
rating_crosstab.shape
X = rating_crosstab.T
X.shape

(8, 101)

In [153]:
SVD = TruncatedSVD(n_components=11, random_state=0)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(8, 8)

In [154]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape
task_names = rating_crosstab.columns
task_list = list(task_names)
task_list
task = task_list.index(('Task1',1000))
task
corr_1 = corr_mat[task]
corr_1.shape
corr_1

array([ 1.        , -0.13381369, -0.14254717, -0.14193453, -0.14425638,
       -0.14667854, -0.13095489, -0.13614685])

In [155]:
list(task_names[(corr_1<=1) & (corr_1 > -0.135)])


[('Task1', 1000), ('Task2', 1010), ('Task7', 1100)]